# Applied Data Science - The Battle of Neighborhoods
The main idea is to leverage Foursquare location data to explore and cluster neighborhoods.

## Introduction/Business problem
Below is provided a concise discussion of the business/data science problem and background, and who would be interested in this project.

### Definition of the problem, related background, and why it is important to solve
To maintain neighborhood compatibility when considering the design of new real estate development or renovation, it is of the essence to retain the character, or look and feel, of the neighborhood of interest. This is achieved, for example, by including a range of venues identifying the neighborhood under consideration. Determining the discriminating venue categories and frequency that distinguish a neighborhood can be a daunting task unless machine learning techniques are enlisted to analyze data that might contain, depending on the neighborhood, 200 venues categories or more. To this aim, clustering is especially useful to discover insights from unlabeled data.

### Where Foursquare location data are needed to solve or execute
Foursquare location data provides information such as the categories of the venues in the neighborhood of choice. Such data are necessary to analyze a neighborhood and determine the most common venues, information that can be fed to a clustering algorithm. By examining the clusters a determination regarding the discriminating venue categories that distinguish each cluster can be made. Moreover, information such as overall rating, number of tips, agree counts, Foursquare users and users' friends, and popular and trending spots around a venue, are available for any further analysis needed by the real estate developers.

### Description of audience and why they would care about the problem
The target audience for this data science problem are real estete developers interested in finding a neighborhood compatible with their design. The target audience cares about this problem because picking the right neighborhood is of the essence for the success of their business venture.

## Data
Below is provided a concise description of the data that will be used to solve the problem and the source of the data.

### Postal codes
Postal codes are needed to _extract_ latitude and longitude coordinates of the neighborhoods of interest. Latitude and longitude coordinate are used to retrieve Foursquare location data containing venue information. **Source:** Wikipedia pages.


### Foursquare
Foursquare is a location data provider. By constructing a specific URL a request can be sent to the Foursquare API to: search for a specific type of venue around a given location, learn more about a specific venue, learn more about a specific Foursquare user, explore a given location, explore trending venues around a given location. In this case we want to _extract_, from Foursquare location data, the unique categories of the venues making up a neighborhood. This will allow for the determination of the mean of the frequency of occurrence of each category. The adopted clustering techniques depends on most common venue data. **Source:** Foursquare Places API.


## Methodology
_Discussion and description of performed exploratory data analysis and inferential statistical testing, and machine learnings used and why._

**Not due yet.**

## Results
_Discussion of the results._

**Not due yet.**

## Discussion
_Discussion of noted observations and recommendations that can me made based on the results._

**Not due yet.**

## Conclusions
_Conclusions of the report._

**Not due yet.**

## Presentation/blog post
_Choice of a presentation or blogpost._
- Just like a report, we need to either prepare a presentation of the project or write a blogpost.
- For a presentation, picture yourself presenting the work to a manager or a client, and build your presentation with that mindset.
- For a blogpost, write a summary of the work in a very elegant and intriguing way. Blogposts are one of the means to showcase work done to potential employers and other data scientists as well.

**Not due yet.**